In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import os
from sys import byteorder
from array import array
from struct import pack
import numpy as np
import librosa
from keras.models import load_model
import pyaudio
import wave

THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000
SILENCE = 30


class GenderRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Gender Recognition App")

        self.model = self.load_model()
        self.create_widgets()

    def create_widgets(self):
        self.file_label = tk.Label(self.root, text="Audio File:")
        self.file_label.pack(pady=10)

        self.select_file_button = tk.Button(self.root, text="Select File", command=self.select_file)
        self.select_file_button.pack(pady=10)

        self.predict_button = tk.Button(self.root, text="Predict Gender", command=self.predict_gender)
        self.predict_button.pack(pady=10)

        self.result_label = tk.Label(self.root, text="")
        self.result_label.pack(pady=10)

        self.quit_button = tk.Button(self.root, text="Quit", command=self.root.destroy)
        self.quit_button.pack(pady=10)

    def select_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("Audio files", "*.wav")])
        self.file_label.config(text="Audio File: " + file_path)
        self.file_path = file_path

    def predict_gender(self):
        if hasattr(self, 'file_path') and os.path.isfile(self.file_path):
            features = self.extract_feature(self.file_path, mel=True).reshape(1, -1)
            male_prob = self.model.predict(features)[0][0]
            female_prob = 1 - male_prob
            gender = "Male" if male_prob > female_prob else "Female"
            result_text = f"Result: {gender}\nProbabilities: Male: {male_prob*100:.2f}% Female: {female_prob*100:.2f}%"
            self.result_label.config(text=result_text)
        else:
            self.result_label.config(text="Please select a valid audio file.")

    @staticmethod
    def load_model():
        return load_model("model.h5")

    @staticmethod
    def is_silent(snd_data):
        return max(snd_data) < THRESHOLD

    @staticmethod
    def extract_feature(file_name, **kwargs):
        X, sample_rate = librosa.core.load(file_name)
        if kwargs.get("mel"):
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            return mel


if __name__ == "__main__":
    root = tk.Tk()
    app = GenderRecognitionApp(root)
    root.mainloop()




1/1 [==============================] - 0s 137ms/step
